In [1]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_pos = pd.read_csv('/content/drive/MyDrive/Technocolab/archive/comments_positive.csv')
df_neg = pd.read_csv('/content/drive/MyDrive/Technocolab/archive/comments_negative.csv')

In [4]:
[df_pos.shape,df_neg.shape]

[(2000000, 15), (2000000, 15)]

In [5]:
df_pos.dropna(inplace=True)
df_neg.dropna(inplace = True)

In [6]:
df_pos.head()

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c092j8m,t1_c092gss,t5_2qh2p,t3_8eyy3,This isn't Twitter: try to comment on the arti...,9582,9582,nraustinii,0,t3_8eyy3,Fucking faggot.,-7526,-7526,Glorificus,0
1,c4imcva,t1_c4im948,t5_2qh1i,t3_t0ynr,"Well, it is exactly what it sounds like. It's ...",9531,9531,Lynfect,0,t3_t0ynr,"Elaborate on this cum box, please.",3841,3841,eeeeevil,0
2,c0s4nfi,t1_c0s4lje,t5_2qh1i,t3_cf1n2,"In soviet Russia, bomb disarms you!",8545,8545,CapnScumbone,0,t3_cf1n2,"I don't live in Russia anymore, and I will not...",621,621,shady8x,0
3,c4ini33,t1_c4incln,t5_2qh1i,t3_t0ynr,"""runin for senitur! #YOLO!""",7430,7430,[deleted],0,t3_t0ynr,This just made me realize that future presiden...,4651,4651,drspg99,0
4,c4imgel,t1_c4ima2e,t5_2qh1i,t3_t0ynr,You step motherfucker.,7173,7173,jbg89,0,t3_t0ynr,I have sex with my step mom when my dad isn't ...,4251,4251,audir8,0


In [7]:
df_pos = df_pos.sample(frac = 0.05, random_state=101)
df_neg = df_neg.sample(frac = 0.05, random_state=101)

## Cleaning The Dataset

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def clean(text, stemming=False, stop_words=True):
    import re
    from string import punctuation
    from nltk.stem import SnowballStemmer
    from nltk.corpus import stopwords
    from nltk import word_tokenize
    
    
    # Empty comment
    if type(text) != str or text=='':
        return ''
    
    # Commence the cleaning!
    urls = r'http(s)*:\/\/(\w|\.)+(\/\w+)*'
    text = re.sub(urls, '', text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are", text)
    text = re.sub("\'ve", " have", text)
    text = re.sub("\'d", " would", text)
    text = re.sub("cant", "can not", text)
    text = re.sub("can\'t", "can not", text)
    text = re.sub("isn\'t", "is not", text)
    text = re.sub("isnt", "is not", text)
    text = re.sub("whats", "what is", text)
    text = re.sub("what\'s", "what is", text)
    text = re.sub("shouldn't", "should not", text, flags=re.IGNORECASE)
    text = re.sub("I'm", "I am", text)
    text = re.sub(":", " ", text)
    # The comments contain \n for line breaks, we need to remove those too
    text = re.sub("\\n", " ", text)
    
    # Special characters
    text = re.sub('\&', " and ", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    
    # Remove punctuation
    text = ''.join([word for word in text if word not in punctuation]).lower()
    
    # If we want to do stemming...
    if stemming:
        sno = SnowballStemmer('english')
        text = ''.join([sno.stem[word] for word in text])
    
    # If we want to remove stop words...
    stops = stopwords.words('english')
    if stop_words:
        text = text.split()
        text = [word for word in text if word not in stops]
        text = ' '.join(text)
    
    return text

In [10]:
df_pos['text'] = df_pos['text'].apply(clean).astype(str)
df_pos['parent_text'] = df_pos['parent_text'].apply(clean).astype(str)

df_neg['text'] = df_neg['text'].apply(clean).astype(str)
df_neg['parent_text'] = df_neg['parent_text'].apply(clean).astype(str)

In [11]:
#Concatinating the data
df = pd.concat([df_pos, df_neg])
df = df.sample(frac=1).reset_index(drop=True)
df.dropna(axis=0, inplace=True)

In [12]:
#combining the text and parent_text
df['combined'] = df[['text','parent_text']].apply(lambda x: ''.join(x),axis = 1)

In [13]:
df['combined'].shape

(199997,)

In [16]:
df_1 = df.sample(frac=0.05,random_state=101)

In [17]:
df_1['combined'].shape

(10000,)

## Splitting the Data

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
...     df_1['combined'], df_1['score'], test_size=0.33, random_state=101)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tf = TfidfVectorizer()

In [22]:
X_train = tf.fit_transform(X_train)
X_test = tf.transform(X_test)

### Model

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [29]:
ln = LinearRegression()
kn = KNeighborsRegressor(n_neighbors=5)
rf = RandomForestRegressor()

In [30]:
ln.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
kn.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [32]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

### Prediction

In [56]:
from sklearn.metrics import mean_squared_error as mse

In [35]:
pred_ln = ln.predict(X_test)

In [40]:
rmse_ln = (mse(y_test,pred_ln))**0.5
rmse_ln

247.62619469046797

In [39]:
pred_kn = kn.predict(X_test)

In [44]:
rmse_kn = (mse(y_test,pred_kn))**0.5
rmse_kn

211.61910995221984

In [42]:
pred_rf = rf.predict(X_test)

In [45]:
rmse_rf = (mse(y_test,pred_rf))**0.5
rmse_rf

209.89148167035162

In [54]:
pred = pd.DataFrame(data=[rmse_rf,rmse_ln,rmse_kn],index=['Random_Forest','Logistic_Regression','KNN'],columns=['RMSE_score'])

In [55]:
pred

,RMSE_score
Random_Forest,209.891482
Logistic_Regression,247.626195
KNN,211.619110


In [ ]:
#So we see thar rf model performs better and so we use rf model for further prediction

### Pickle

In [59]:
import pickle

In [65]:
pk = open('model.pkl','wb')
pickle.dump(ln,pk)
pickle.dump(tf, pk)